## Dataset Pre-processing
### Download Beauty Dataset from [Amazon 2018](https://nijianmo.github.io/amazon/index.html)

In [8]:
!python data/data_process.py --file_path data/Beauty/All_Beauty_5.json.gz --output_path data/Beauty

5269it [00:00, 109083.04it/s]
100%|█████████████████████████████████████| 991/991 [00:00<00:00, 434649.72it/s]


## Load Dependencies and Hyper-Parameters

In [1]:
import torch

from src.models import model_factory
from src.dataloaders import dataloader_factory
from src.datasets import dataset_factory
from src.trainers import trainer_factory
from src.utils.utils import *
from src.utils.options import parser

In [2]:
args = parser.parse_args([])

args.data_path = 'data/Beauty'
args.num_epochs = 100
args.trm_max_len = 50

ckpt_root = setup_train(args)

Folder created: /home/zhankui/FastRec/experiments/test_2021-11-11_32
{'adam_epsilon': 1e-06,
 'best_metric': 'NDCG@10',
 'data_path': 'data/Beauty',
 'dataloader_code': 'sasrec',
 'dataloader_random_seed': 0.0,
 'dataset_code': 'item',
 'dataset_split_seed': 98765,
 'device': 'cpu',
 'device_idx': '0',
 'experiment_description': 'test',
 'experiment_dir': 'experiments',
 'global_epochs': 1000,
 'local_epochs': 10,
 'lr': 0.001,
 'metric_ks': [5,
               10,
               20],
 'model_code': 'sasrec',
 'model_init_seed': 0,
 'num_epochs': 100,
 'num_gpu': 1,
 'optimizer': 'Adam',
 'split': 'leave_one_out',
 'subset_size': 1000,
 'test_batch_size': 64,
 'test_negative_sample_size': 100,
 'test_negative_sampler_code': 'random',
 'test_negative_sampling_seed': 98765,
 'train_batch_size': 64,
 'train_negative_sample_size': 100,
 'train_negative_sampler_code': 'random',
 'train_negative_sampling_seed': 0,
 'trainer_code': 'sasrec_sample',
 'trm_att_dropout': 0.2,
 'trm_dropout': 0.2,

## Build Dataset

In [3]:
dataset = dataset_factory(args)
train_loader, val_loader, test_loader, dataset = dataloader_factory(args, dataset)

## Model Setup
We load [SASRec Model](https://arxiv.org/abs/1808.09781) for sequential recommendation

In [4]:
model = model_factory(args)
if args.load_pretrained_weights is not None:
    print("weights loading from %s ..." % args.load_pretrained_weights)
    model = load_pretrained_weights(model, args.load_pretrained_weights)
print("Model size:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print(model)

Model size: 37950
SASRecModel(
  (loss): BCEWithLogitsLoss()
  (item_emb): Embedding(87, 50, padding_idx=86)
  (pos_emb): Embedding(50, 50)
  (emb_dropout): Dropout(p=0.2, inplace=False)
  (attention_layernorms): ModuleList(
    (0): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
    (1): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0): MultiHeadAttention(
      (linear_layers): ModuleList(
        (0): Linear(in_features=50, out_features=50, bias=True)
        (1): Linear(in_features=50, out_features=50, bias=True)
        (2): Linear(in_features=50, out_features=50, bias=True)
      )
      (output_linear): Linear(in_features=50, out_features=50, bias=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): MultiHeadAttention(
      (linear_layers): ModuleList(
        (0): Linear(in_features=50, out_features=50, bias=True)
        (1): Linear(in_features=50, out_features=50, bias=True)
        (2): Linear(in_feature

## Model Training

In [5]:
trainer = trainer_factory(args, model, train_loader, val_loader, test_loader, ckpt_root, dataset.data)
trainer.train()

TEST N@5 0.40320: , N@10 0.52204: , N@20 0.53289: , R@5 0.41507: , R@10 0.77886: , R@20 0.82110: , M 0.45544: , AUC 0.86572: , loss 0.00000: 100%|██████████| 16/16 [00:02<00:00,  6.83it/s]
  0%|          | 0/16 [00:00<?, ?it/s]/home/zhankui/FastRec/src/trainers/base.py:85: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 5.0)
Epoch 10, loss 0.25110 : 100%|██████████| 16/16 [00:00<00:00, 32.41it/s]
VAL N@5 0.84126: , N@10 0.84721: , N@20 0.84917: , R@5 0.96087: , R@10 0.97845: , R@20 0.98633: , M 0.80434: , AUC 0.98443: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 167.21it/s]
TEST N@5 0.59939: , N@10 0.60403: , N@20 0.60663: , R@5 0.82201: , R@10 0.83569: , R@20 0.84649: , M 0.52947: , AUC 0.88184: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 147.03it/s]
Epoch 11, loss 0.23759 :   0%|          | 0/16 [00:00<?, ?it/s]

Update Best NDCG@10 Model


Epoch 20, loss 0.14201 : 100%|██████████| 16/16 [00:00<00:00, 31.52it/s]
VAL N@5 0.87817: , N@10 0.88217: , N@20 0.88457: , R@5 0.96472: , R@10 0.97741: , R@20 0.98718: , M 0.85075: , AUC 0.98706: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 135.42it/s]
TEST N@5 0.70221: , N@10 0.70414: , N@20 0.70797: , R@5 0.82683: , R@10 0.83269: , R@20 0.84740: , M 0.66374: , AUC 0.89180: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 149.72it/s]
Epoch 21, loss 0.17444 :   0%|          | 0/16 [00:00<?, ?it/s]

Update Best NDCG@10 Model


Epoch 30, loss 0.12022 : 100%|██████████| 16/16 [00:00<00:00, 31.09it/s]
VAL N@5 0.91585: , N@10 0.91930: , N@20 0.92105: , R@5 0.97064: , R@10 0.98138: , R@20 0.98822: , M 0.89923: , AUC 0.98931: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 149.42it/s]
TEST N@5 0.73060: , N@10 0.73415: , N@20 0.73643: , R@5 0.83178: , R@10 0.84258: , R@20 0.85137: , M 0.70001: , AUC 0.89340: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 153.99it/s]
Epoch 31, loss 0.09463 :   0%|          | 0/16 [00:00<?, ?it/s]

...

Epoch 60, loss 0.07231 : 100%|██████████| 16/16 [00:00<00:00, 29.06it/s]
VAL N@5 0.92803: , N@10 0.93295: , N@20 0.93570: , R@5 0.95990: , R@10 0.97455: , R@20 0.98529: , M 0.92024: , AUC 0.98860: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 149.67it/s]
TEST N@5 0.78870: , N@10 0.79101: , N@20 0.79352: , R@5 0.83263: , R@10 0.83953: , R@20 0.84936: , M 0.77740: , AUC 0.88458: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 177.03it/s]
Epoch 61, loss 0.07173 :   0%|          | 0/16 [00:00<?, ?it/s]

Update Best NDCG@10 Model


Epoch 70, loss 0.06383 : 100%|██████████| 16/16 [00:00<00:00, 32.31it/s]
VAL N@5 0.92656: , N@10 0.93167: , N@20 0.93343: , R@5 0.96387: , R@10 0.97949: , R@20 0.98633: , M 0.91658: , AUC 0.98981: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 173.63it/s]
TEST N@5 0.75532: , N@10 0.75785: , N@20 0.76045: , R@5 0.83373: , R@10 0.84154: , R@20 0.85229: , M 0.73168: , AUC 0.87429: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 166.57it/s]
Epoch 80, loss 0.06424 : 100%|██████████| 16/16 [00:00<00:00, 27.90it/s]
VAL N@5 0.93558: , N@10 0.94134: , N@20 0.94259: , R@5 0.96484: , R@10 0.98242: , R@20 0.98730: , M 0.92857: , AUC 0.99037: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 176.33it/s]
TEST N@5 0.78630: , N@10 0.78867: , N@20 0.79188: , R@5 0.83172: , R@10 0.83855: , R@20 0.85131: , M 0.77446: , AUC 0.87928: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 174.17it/s]
Epoch 81, loss 0.04782 :   0%|          | 0/16 [00:00<?, ?it/s]

Update Best NDCG@10 Model


Epoch 90, loss 0.06131 : 100%|██████████| 16/16 [00:00<00:00, 28.14it/s]
VAL N@5 0.92833: , N@10 0.93350: , N@20 0.93654: , R@5 0.95996: , R@10 0.97559: , R@20 0.98730: , M 0.92077: , AUC 0.98996: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 173.20it/s]
TEST N@5 0.78201: , N@10 0.78463: , N@20 0.78689: , R@5 0.82964: , R@10 0.83745: , R@20 0.84624: , M 0.76937: , AUC 0.87221: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 161.15it/s]
Epoch 100, loss 0.05626 : 100%|██████████| 16/16 [00:00<00:00, 31.20it/s]
VAL N@5 0.92916: , N@10 0.93393: , N@20 0.93642: , R@5 0.96094: , R@10 0.97559: , R@20 0.98535: , M 0.92144: , AUC 0.98948: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 130.66it/s]
TEST N@5 0.78709: , N@10 0.78999: , N@20 0.79207: , R@5 0.83055: , R@10 0.83934: , R@20 0.84715: , M 0.77586: , AUC 0.87407: , loss 0.00000: 100%|██████████| 16/16 [00:00<00:00, 154.18it/s]


## Model Test and Saving

In [ ]:
# model testing and saving
trainer.test()
trainer.logger_service.complete({'state_dict': (trainer._create_state_dict())})
trainer.writer.close()

## Export ONNX Model 
- Install ONNX with `pip install ONNX`
- Install ONNX Runtime ()ORT with `pip install onnxruntime` or `pip install onnxruntime-gpu`

In [6]:
model.eval()
for batch in trainer.test_loader:
    break
users, seqs, candidates, labels, length = batch
args = (seqs, candidates, length)
torch_out = model(*args)

### Export Our Model with ONNX (Ver. 11)

In [7]:
torch.onnx.export(model,               # model being run
                  args,                         # model input (or a tuple for multiple inputs)
                  "demo_model.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['seqs', 'candidates', 'length'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

/home/zhankui/.local/lib/python3.6/site-packages/torch/onnx/utils.py:1190: UserWarning: Provided key input for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
/home/zhankui/FastRec/src/models/onnx_support.py:55: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))
/home/zhankui/.local/lib/python3.6/site-packages/torch/onnx/symbolic_opset9.py:2624: UserWarning: Exporting aten::index operator of advanced indexing in opset 11 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect res

### Load Our ONNX Model for Checking

In [8]:
import onnx

onnx_model = onnx.load("demo_model.onnx")
onnx.checker.check_model(onnx_model)

### Use ONNXRuntime to Run our Model

In [9]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("demo_model.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

/home/zhankui/anaconda3/envs/bert4rec/lib/python3.6/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:353: UserWarning: Deprecation warning. This ORT build has ['CUDAExecutionProvider', 'CPUExecutionProvider'] enabled. The next release (ORT 1.10) will require explicitly setting the providers parameter (as opposed to the current behavior of providers getting set/registered by default based on the build flags) when instantiating InferenceSession.For example, onnxruntime.InferenceSession(..., providers=["CUDAExecutionProvider"], ...)
  "based on the build flags) when instantiating InferenceSession."


In [10]:
# compute ONNX Runtime output prediction
ort_inputs = {'seqs': to_numpy(seqs), 'candidates': to_numpy(candidates), 'length': to_numpy(length)}
ort_outs = ort_session.run(None, ort_inputs)

In [11]:
# compare ONNX Runtime and PyTorch results
print(np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05))

None


### You can also use `onnx_tensorrt` to use `TensorRT` backend

- Here we use TensorRT 8.2.0, please follow [onnx_tensorrt](https://github.com/onnx/onnx-tensorrt) master branch to install the related dependencies. Or you can use `onnx2trt` to export this `.onnx` to `.trt` engine.

In [ ]:
import onnx
import onnx_tensorrt.backend as backend

onnx_model = onnx.load("demo_model.onnx")
engine = backend.prepare(onnx_model, device='CUDA:0')

## Model Serving
We deploy a model with [faiss](https://github.com/facebookresearch/faiss) to accelerate the maximal inner product search. 

In [14]:
import os
import faiss

model = model_factory(args)
model = load_pretrained_weights(model, os.path.join(ckpt_root, 'models', 'best_acc_model.pth'))
model = model.eval()

In [15]:
import numpy as np
d = args.trm_hidden_dim                                 # dimension
nb = model.item_emb.weight.size(0)                      # item pool size
xb = model.item_emb.weight.data.cpu().numpy()           # item embeddings table

In [16]:
index = faiss.index_factory(d, 'HNSW32', faiss.METRIC_INNER_PRODUCT)   # build the index
index.add(xb)                                                          # add vectors to the index

### Model Input Example

In [41]:
x = torch.LongTensor([dataset.train[0]])                # input example
l = torch.LongTensor([len(dataset.train[0])-1])         # length

In [23]:
xq = model(x, length=l, mode='serving').detach().cpu()

### Exact Search via Naive Matmul

In [34]:
k = 10

V, I = torch.topk((torch.Tensor(xb) @ xq.squeeze()), k)
print(I)

tensor([63, 29, 28, 61, 36,  6, 49, 62, 12, 39])


### Approximate Search Via Faiss

In [40]:
V, I = index.search(xq.numpy(), k)
print(I)

[[63 29 28 36 61  6 62 12 49 39]]
